In [2]:
import os
import pathlib
from urllib.request import urlretrieve, urlopen
from time import sleep
import math as math
import quandl
import pandas as pd
import numpy as np
import boto3

In [3]:
quandl.ApiConfig.api_key = os.environ['QUANDL_API_KEY']

In [4]:
cme_metadata = pd.read_csv('/code/data/CME_metadata.csv')
cme_metadata.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=https...",2018-12-06 01:06:06,2015-02-12,2018-12-05


In [5]:
cme_futures = pd.read_csv('/code/data/CME_futures.csv')

#cme_futures.set_index('Futures Code', inplace=True)
cme_futures.head(50)

,Futures Symbol,Exchange,Name,Hold Roll Cycle,Priced Roll Cycle,CME Months,Start Year
0,AD,CME,Australian Dollar Futures,HMUZ,HMUZ,HMUZ,1987
1,BO,CBT,Soybean Oil Futures,Z,FHKNQUVZ,FHKNQUVZ,1962
2,BP,CME,British Pound Futures,HMUZ,HMUZ,HMUZ,1976
3,C,CBT,Corn Futures,Z,HKNUZ,HKNUZ,1960
4,CD,CME,Canadian Dollar Futures,HMUZ,HMUZ,HMUZ,1978
5,CL,NYMEX,Crude Oil Futures,Z,FGHJKMNQUVXZ,FGHJKMNQUVXZ,1984
6,EC,CME,Euro FX Futures,HMUZ,HMUZ,HMUZ,1999
7,ED,CME,Eurodollar Futures,HMUZ,HMUZ,HJKMNQUZ,1982
8,ES,CME,E-mini S&P 500 Futures,HMUZ,HMUZ,HMUZ,1999
9,FC,CME,Feeder Cattle Futures,FHJKQUVX,FHJKQUVX,FHJKQUVX,1982


In [7]:
pd.options.display.max_rows = 999
futures_codes = cme_futures['Futures Symbol'].tolist()
contract_codes = []

for futures_code in futures_codes:
    length = (cme_metadata['code'].str.len() == len(futures_code) + 5)
    starts_with = cme_metadata['code'].str.startswith(futures_code)
    results = cme_metadata[length & starts_with]['code'].tolist()
    contract_codes.extend(results)

len(contract_codes)

9320

In [44]:
def download_contract_from_quandl(exchange_code, contract_code, data_directory):
    API_URL = f'http://www.quandl.com/api/v3/datasets/{exchange_code}/{contract_code}.csv' \
            f'?auth_token={quandl.ApiConfig.api_key}&order=asc'
    
    if len(contract_code) == 6:
        futures_code = contract_code[0:1]
    elif len(contract_code) == 7:
        futures_code = contract_code[0:2]
    else:
        futures_code = contract_code[0:3]
    CONTRACT_DATA_DIRNAME = data_directory / 'raw_csv' / 'futures' / f'{futures_code}' / 'daily'
    
    CONTRACT_DATA_DIRNAME.mkdir(parents=True, exist_ok=True)
    os.chdir(CONTRACT_DATA_DIRNAME)
    urlretrieve(API_URL, f'{contract_code}.csv')
    sleep(0.3) # API Rate Limit is 300 calls per 10 seconds, 2000 calls per 10 minutes
    
DATA_DIRNAME = pathlib.Path('/code').resolve() / 'data'
for contract_code in contract_codes:
    print (f'Downloading Contract: {contract_code}')
    download_contract_from_quandl('CME', contract_code, DATA_DIRNAME)

In [2]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

financial-data-monitor
map-sites
map-sites-media
map-sites-static
single-prop-sites
toolkmit


In [3]:
p = pathlib.Path('/code').resolve() / 'data'
cme_metadata = pd.read_csv(p.resolve() / 'CME_metadata.csv')
cme_metadata.head()

,code,name,description,refreshed_at,from_date,to_date
0,0DF2015,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-12-28
1,0DF2016,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-06-29 17:33:32,2014-02-18,2016-02-02
2,0DF2017,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2017-02-03 01:05:31,2014-11-21,2017-02-02
3,0DF2018,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=http:...",2018-02-03 01:07:36,2014-11-21,2018-02-02
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,"For contract details, please see <a href=https...",2018-12-06 01:06:06,2015-02-12,2018-12-05


In [6]:
s3 = boto3.client('s3')
s3.upload_file('/code/data/CME_metadata.csv',
               'financial-data-monitor', 
               'data/CME_metadata.csv')

In [8]:
exchange_code = 'CME'
contract_code = 'ESZ2018'

API_URL = f'http://www.quandl.com/api/v3/datasets/{exchange_code}/{contract_code}.csv' \
            f'?auth_token={quandl.ApiConfig.api_key}&order=asc'
API_URL

'http://www.quandl.com/api/v3/datasets/CME/ESZ2018.csv?auth_token=TFddxxSjUbLtV482Dq7Q&order=asc'

In [10]:
s3 = boto3.client('s3')
with urlopen(API_URL) as data:
    s3.upload_fileobj(data, 'financial-data-monitor', f'{contract_code}.csv')

In [11]:
def upload_quandl_contract_to_s3(exchange_code, contract_code, boto_client):
    if len(contract_code) == 6:
        futures_code = contract_code[0:1]
    elif len(contract_code) == 7:
        futures_code = contract_code[0:2]
    else:
        futures_code = contract_code[0:3]
        
    API_URL = f'http://www.quandl.com/api/v3/datasets/{exchange_code}/{contract_code}.csv' \
            f'?auth_token={quandl.ApiConfig.api_key}&order=asc'
    
    FILENAME = f'data/raw_csv/futures/{futures_code}/daily/{contract_code}.csv'
    with urlopen(API_URL) as data:
        boto_client.upload_fileobj(data, 'financial-data-monitor', FILENAME)
    sleep(0.3) # API Rate Limit is 300 calls per 10 seconds, 2000 calls per 10 minutes

In [12]:
upload_quandl_contract_to_s3(exchange_code, contract_code, s3)